## 2.3 통계 기반 기법

### 2.3.1 파이썬으로 말뭉치 전처리하기

In [1]:
text = 'You say goodbye and I say hello.'

In [2]:
text = text.lower()
text = text.replace('.', ' .')
text

'you say goodbye and i say hello .'

In [3]:
words = text.split(' ')
words

['you', 'say', 'goodbye', 'and', 'i', 'say', 'hello', '.']

In [4]:
word_to_id = {}
id_to_word = {}

for word in words:
    if word not in word_to_id:
        new_id = len(word_to_id)
        word_to_id[word] = new_id
        id_to_word[new_id] = word

In [6]:
word_to_id

{'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5, '.': 6}

In [7]:
id_to_word

{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}

In [8]:
import numpy as np
corpus = [word_to_id[w] for w in words]
corpus = np.array(corpus)
corpus

array([0, 1, 2, 3, 4, 1, 5, 6])

* 위의 내용이 common/util.py에 preprocess() 함수로 구현됨

### 2.3.2 단어의 분산 표현

distributional representation: 단어의 의미를 정확하게 파악할 수 있는 벡터 표현  
`NOTE`: 고정 길이의 밀집 벡터 (대부분의 원소가 0이 아닌 실수인 벡터)로 표현  
이를 어떻게 구축할 것인가?

### 2.3.3 분포 가설

distributional hypothesis: 단어의 의미는 주변 단어에 의해 형성된다.  
단어 자체에는 의미가 없고, 그 단어가 사용된 '맥락 context'이 의미를 형성한다. 즉, 주목하는 단어 주변에 놓은 단어  
맥락의 크기 (주변 단어를 몇 개나 포함할지)를 '윈도우 크기 window size'라고 한다. - 상황에 따라 왼쪽, 오른쪽, 문장의 시작과 끝을 고려  

### 2.3.4 동시발생 행렬

모든 단어에 대해 동시발생하는 단어를 표에 정리 - 행렬의 형태를 띠므로 co-occurrence matrix라고 함

In [1]:
import sys
sys.path.append('..')
import numpy as np
from common.util import preprocess, create_co_matrix, most_similar

In [3]:
text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

In [4]:
print(corpus)

[0 1 2 3 4 1 5 6]


In [5]:
print(id_to_word)

{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


* common/util.py에 create_co_matrix() 함수로 동시발생 행렬을 만듦

### 2.3.5 벡터 간 유사도

코사인 유사도를 자주 이용:
$$ similarity(\mathbf{x, y})\, =\, \frac{\mathbf{x\cdot y}}{\mathbf{||x||||y||}}\, =\, \frac{x_1y_1+...+x_ny_n}{\sqrt{x_1^2+...+x_n^2}\sqrt{y_1^2+...+y_n^2}}$$

벡터를 정규화하고 내적을 구하는 것  
두 벡터가 가리키는 방향이 얼마나 비슷한가?  
-1: 완전히 반대 방향, 1: 완전히 같은 방향  

* common/util.py에 cos_similarity() 함수로 구현됨

### 2.3.6 유사 단어의 랭킹 표시

어떤 단어가 검색어로 주어지면, 그 검색어와 비슷한 단어를 유사도 순으로 출력하는 함수  
* common/util.py에 most_similar() 함수로 구현

In [6]:
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)

most_similar('you', word_to_id, id_to_word, C, top=5)


[query] you
 goodbye: 0.7071067691154799
 i: 0.7071067691154799
 hello: 0.7071067691154799
 say: 0.0
 and: 0.0


## 2.4 통계 기반 기법 개선하기

### 2.4.1 상호정보량

동시 발생의 횟수만 본다면 영어의 관사가 고빈도 단어라 우리의 주제 단어와 강한 관련성을 갖는다고 평가할 수 있다.  
이를 해결하기 위해 **점별 상호정보량 pointwise mutual information (PMI)**라는 척도를 사용  

$$ PMI(x,y)\, =\, \log_2\frac{P(x,y)}{P(x)P(y)} $$  

값이 높을수록 관련성이 높다.  

두 단어의 동시발생 횟수가 0이면 $\log_{2}0\, =\, -\infty$가 된다.  
따라서, **양의 상호정보량 Positive PMI**를 사용

$$ PPMI(x, y)\, =\, max(0, PMI(x, y)) $$

PMI가 음수일 때는 0으로 취급
* common/util.py에 ppmi(C, verbose=False, eps=1e-8)로 구현

말뭉치의 어휘 수가 증가함에 따라 각 단어 벡터의 차원 수도 증가한다.  
원소 대부분이 0이다.  각 원소의 '중요도'가 낮다.  
노이즈에 약하고 견고하지 못하다는 약점  

이를 대처하고자 벡터의 차원 감소 이용

### 2.4.2 차원 감소

dimensionality reduction - 중요한 정보는 최대한 유지하면서 줄이는 게 핵심  
`희소 행렬` (원소 대부분이 0인 행렬)에서 중요한 축을 발견하여 `밀집 벡터`로 변환: 우리가 원하는 단어의 분산 표현  

특이값분해 Singular value decomposition (SVD):  

$$ \mathbf{X}\, =\, \mathbf{USV^T} $$

$U$와 $V$는 직교행렬, $S$는 대각행렬  
직교행렬은 어떠한 공간의 축 (기저)을 형성 - $U$를 '단어 공간'으로 취급  
$S$는 대각행렬로, 대각성분에는 '특이값'이 큰 순서로 나열 - 해당 축의 중요도라고 간주, 낮은 중요도의 원소를 깎아낼 수 있다.  


### 2.4.3 SVD에 의한 차원 감소

넘파이의 `linalg` 모듈 이용,  
count_method_small.py 참고  

### 2.4.4 PTB 데이터셋

Penn Treebank: 주어진 기법의 품질을 측정하는 벤치마크로 자주 이용  

### 2.4.5 PTB 데이터셋 평가

count_method_big.py 참고
sklearn의 randomized_svd() 메서드 사용 - truncated SVD